# Direct Male Descent

The following code was the first I wrote after extracting the character data from the save file and before adding the arrays in cell 3 of the first script and using that to get descent through both male and female descendents. Because "fat" is used in the connectToField of the $graphLoopup it will get all of Charlemagne's children but will only get grandchildren where there father is one of Charlemagne's children. In script 3, using the "parents" or "rparents" array instead solves this.

In [1]:
from pymongo import MongoClient
import pandas as pd
import datetime

In [2]:
client = MongoClient()
characters = client.ck2.characters

## Direct Descendents of Charlemagne

In [3]:
pipeline = [{
    "$match" : { "_id" : 6392} #Charlemagne
},
    {
    
      "$graphLookup": {
      "from": "characters",
      "startWith": "$_id",
      "connectFromField": "_id",
      "connectToField": "fat",
      "as": "descendent"
   }},
    {
        "$project" : { "bn" : 1, "descendents" : {"$size" : "$descendent"} }
    }
]

In [4]:
karl = characters.aggregate(pipeline) 

In [5]:
for char in karl:
    print(char)

{'_id': 6392, 'bn': 'Charles', 'descendents': 430}


## Direct Descendents Within 2 Generations

In [6]:
pipeline = [
    {
        "$match" : { "_id" : 6392}
    },
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "fat",
            "as": "descendent",
            "maxDepth": 1,
            "depthField": "generation"
        }
    },
    {"$unwind" : "$descendent" },
    {"$sort" : {"descendent.b_d" : 1}}
]

In [7]:
karl = characters.aggregate(pipeline)

In [8]:
for char in karl:
    print(char)
    print("\n\n")

{'_id': 6392, 'bn': 'Charles', 'hist': 1, 'b_d': datetime.datetime(742, 1, 2, 0, 0), 'd_d': datetime.datetime(801, 10, 19, 0, 0), 'c_d': 'death_murder_unknown_fall', 'killer': 190454, 'fat': 131722, 'mot': 190429, 'cul': 'old_frankish', 'dnt': 25061, 'spawned_bastards': 1, 'spouse': [509092, 524280], 'parents': [131722, 190429], 'rparents': [131722, 190429], 'descendent': {'_id': 194169, 'bn': 'Amaudru', 'hist': 1, 'fem': 1, 'b_d': datetime.datetime(767, 1, 1, 0, 0), 'd_d': datetime.datetime(821, 7, 18, 0, 0), 'c_d': 'death_trait', 'lge': 527132, 'fat': 6392, 'mot': 194168, 'dnt': 25061, 'spouse': [190229], 'parents': [6392, 194168], 'rparents': [6392, 194168], 'generation': 0}}



{'_id': 6392, 'bn': 'Charles', 'hist': 1, 'b_d': datetime.datetime(742, 1, 2, 0, 0), 'd_d': datetime.datetime(801, 10, 19, 0, 0), 'c_d': 'death_murder_unknown_fall', 'killer': 190454, 'fat': 131722, 'mot': 190429, 'cul': 'old_frankish', 'dnt': 25061, 'spawned_bastards': 1, 'spouse': [509092, 524280], 'parent

## Total Generations

In this case depthField adds a field called generation which increments each step it takes from the starting node. Charlemagne's children have a value of 0, his grandchildren have a value of 1 and so on. In this game Charlemagne has a 23 x Great Grandchild.

In [9]:
pipeline = [
    {
        "$match" : { "_id" : 6392}
    },
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "fat",
            "as": "descendent",
            #"maxDepth": 1,
            "depthField": "generation"
        }
    },
    {"$unwind" : "$descendent" },
    {"$group" : {"_id" : None, "maxGen" : {"$max" : "$descendent.generation"} }}
]

In [10]:
karl = characters.aggregate(pipeline)

In [11]:
for char in karl:
    print(char)
    print("\n\n")

{'_id': None, 'maxGen': 25}





## Ancestors of a character

Same calculation above but in reverse. Starting node is Pierre, an 8 x Grandson of Charlemagne. His father is at generation 0 and his lineage can be traced back 17 generations in total to Arnold, born in 557.

In [12]:
pipeline = [{
    "$match" : { "_id" : 712365} #Pierre, 8 x Great grandson of Charlemagne
},
    {
    
      "$graphLookup": {
      "from": "characters",
      "startWith": "$fat",
      "connectFromField": "fat",
      "connectToField": "_id",
      "as": "ancestor",
      "depthField": "generation"
   }},
    {"$unwind" : "$ancestor"},
    {"$sort" : { "ancestor.generation": -1 }},
    {"$project" : {"_id" : 0, "ancestor" : 1}}
]

In [13]:
karl = characters.aggregate(pipeline)

In [14]:
for char in karl:
    print(char)
    print("\n")

{'ancestor': {'_id': 190412, 'bn': 'Arnold', 'hist': 1, 'b_d': datetime.datetime(557, 1, 1, 0, 0), 'd_d': datetime.datetime(602, 1, 1, 0, 0), 'dnt': 25061, 'parents': [None, None], 'rparents': [None, None], 'generation': 16}}


{'ancestor': {'_id': 190413, 'bn': 'Arnulf', 'hist': 1, 'b_d': datetime.datetime(580, 1, 1, 0, 0), 'd_d': datetime.datetime(640, 7, 18, 0, 0), 'fat': 190412, 'dnt': 25061, 'parents': [190412, None], 'rparents': [190412, None], 'generation': 15}}


{'ancestor': {'_id': 190417, 'bn': 'Ansegisel', 'hist': 1, 'b_d': datetime.datetime(612, 1, 1, 0, 0), 'd_d': datetime.datetime(662, 1, 1, 0, 0), 'fat': 190413, 'dnt': 25061, 'spouse': [91404], 'parents': [190413, None], 'rparents': [190413, None], 'generation': 14}}


{'ancestor': {'_id': 131720, 'bn': 'Pepin', 'hist': 1, 'b_d': datetime.datetime(636, 1, 2, 0, 0), 'd_d': datetime.datetime(714, 1, 17, 0, 0), 'fat': 190417, 'mot': 91404, 'dnt': 25061, 'spouse': [190420], 'parents': [190417, 91404], 'rparents': [190417, 9

## All Direct Descendents of Charlemagne

In [15]:
pipeline = [{
    "$match" : { "_id" : 6392}
},
    {
    
      "$graphLookup": {
      "from": "characters",
      "startWith": "$_id",
      "connectFromField": "_id",
      "connectToField": "fat",
      "as": "descendent",
      "depthField": "generation"
   }},
    {"$unwind" : "$descendent"},
    {"$sort" : { "descendent.b_d": -1 }},
    {"$project" : {"_id" : 0, "descendent" : 1}}
]

In [16]:
karl = characters.aggregate(pipeline)

In [17]:
for char in karl:
    print(char)
    print("\n\n")

{'descendent': {'_id': 1045441, 'bn': 'Cúánán', 'b_d': datetime.datetime(1459, 5, 21, 0, 0), 'fat': 1014184, 'mot': 1011996, 'rel': 'catholic', 'cul': 'pictish', 'dnt': 9230, 'emp': 1014184, 'host': 1014184, 'parents': [1014184, 1011996], 'rparents': [1014184, 1011996], 'generation': 23}}



{'descendent': {'_id': 1043726, 'bn': 'Adalmode', 'fem': 1, 'b_d': datetime.datetime(1457, 6, 5, 0, 0), 'fat': 995217, 'mot': 1041436, 'cul': 'frankish', 'dnt': 25061, 'emp': 995217, 'host': 995217, 'parents': [995217, 1041436], 'rparents': [995217, 1041436], 'generation': 24}}



{'descendent': {'_id': 1042687, 'bn': 'Alf', 'b_d': datetime.datetime(1456, 2, 19, 0, 0), 'fat': 1017356, 'mot': 1034002, 'cul': 'welsh', 'dnt': 10324999, 'emp': 1017356, 'host': 1017356, 'parents': [1017356, 1034002], 'rparents': [1017356, 1034002], 'generation': 23}}



{'descendent': {'_id': 1041004, 'bn': 'Halsten', 'b_d': datetime.datetime(1454, 3, 9, 0, 0), 'fat': 1018658, 'mot': 1036484, 'cul': 'welsh', 'dnt': 1032

{'descendent': {'_id': 687504, 'bn': 'Amelie', 'fem': 1, 'b_d': datetime.datetime(1023, 12, 15, 0, 0), 'd_d': datetime.datetime(1047, 1, 21, 0, 0), 'c_d': 'death_childbirth', 'lge': 682897, 'fat': 669329, 'mot': 672434, 'dnt': 10303571, 'spouse': [686990], 'parents': [669329, 672434], 'rparents': [669329, 672434], 'generation': 9}}



{'descendent': {'_id': 686815, 'bn': 'Raynaud', 'nick': 'nick_the_pious', 'b_d': datetime.datetime(1023, 1, 10, 0, 0), 'd_d': datetime.datetime(1094, 5, 4, 0, 0), 'c_d': 'death_trait', 'fat': 669002, 'mot': 681305, 'cul': 'occitan', 'dnt': 25061, 'parents': [669002, 681305], 'rparents': [669002, 681305], 'generation': 9}}



{'descendent': {'_id': 686471, 'bn': 'Aldebert', 'b_d': datetime.datetime(1022, 7, 15, 0, 0), 'd_d': datetime.datetime(1077, 10, 24, 0, 0), 'c_d': 'death_trait', 'lge': 702062, 'fat': 669329, 'mot': 672434, 'cul': 'old_frankish', 'dnt': 10303571, 'spouse': [688340, 696563, 702062], 'parents': [669329, 672434], 'rparents': [669329, 672

## Find only descendents who have been killed

In [18]:
pipeline = [{
    "$match" : { "_id" : 6392}
},
    {
    
      "$graphLookup": {
      "from": "characters",
      "startWith": "$_id",
      "connectFromField": "_id",
      "connectToField": "fat",
      "as": "descendent",
      "depthField": "generation"
   }},
    {"$unwind" : "$descendent"},
    {"$sort" : { "descendent.b_d": -1 }},
    {"$match" : {"descendent.killer" : {"$exists" : True}}}, #Excludes all descendents who don't have the killer field set
    {"$project" : {"_id" : 0, "descendent" : 1}}
]

In [19]:
karl = characters.aggregate(pipeline)

In [20]:
for char in karl:
    print(char)
    print("\n")

{'descendent': {'_id': 1029240, 'bn': 'Tiburge', 'fem': 1, 'b_d': datetime.datetime(1440, 6, 12, 0, 0), 'd_d': datetime.datetime(1455, 5, 21, 0, 0), 'c_d': 'death_execution_eaten', 'killer': 1017458, 'lge': 1017458, 'fat': 1002826, 'rfat': 985060, 'mot': 995941, 'cul': 'old_frankish', 'dnt': 25061, 'parents': [1002826, 995941], 'rparents': [985060, 995941], 'generation': 23}}


{'descendent': {'_id': 1021195, 'bn': 'Åke', 'b_d': datetime.datetime(1431, 5, 1, 0, 0), 'd_d': datetime.datetime(1450, 4, 16, 0, 0), 'c_d': 'death_battle', 'killer': 1012771, 'lge': 1014184, 'fat': 994796, 'mot': 1008532, 'rel': 'catholic', 'cul': 'pictish', 'dnt': 9230, 'parents': [994796, 1008532], 'rparents': [994796, 1008532], 'generation': 22}}


{'descendent': {'_id': 988272, 'bn': 'Ida', 'fem': 1, 'b_d': datetime.datetime(1392, 6, 4, 0, 0), 'd_d': datetime.datetime(1441, 5, 16, 0, 0), 'c_d': 'death_murder', 'killer': 980989, 'lge': 980989, 'fat': 970238, 'mot': 968635, 'cul': 'frankish', 'dnt': 25061, 's

## Dynasties Married Into By Direct Descendents

This cell groups the dynasties that Charlemagne's descendents have married into. This would be his children, his grandchildren through his sons, his great grandchildren through his grandsons etc. Most marriages have been within the family (this is double counted?), the Alachisling dynasty would rule France and Italy, the Abbonid's would also rule France and through their marriage into the Karling family the Irish Eóganacht-Locha Léin would very briefly rule Aquitaine. 

A list of all families to rule a Kingdom or Empire level title and the titles they held is shown in notebook 7.1 Titles per Dynasty.

In [21]:
pipeline = [{
    "$match" : { "_id" : 6392}
},
    {
    
      "$graphLookup": {
      "from": "characters",
      "startWith": "$_id",
      "connectFromField": "_id",
      "connectToField": "fat",
      "as": "descendent",
      "depthField": "generation"
   }},
    {"$unwind" : "$descendent"},
    {"$unwind" : "$descendent.spouse"},
    {"$lookup" :
     {
            "from" : "characters",
            "localField" : "descendent.spouse",
            "foreignField" : "_id",
            "as" : "spouse_data"
        }
    },
    {"$unwind" : "$spouse_data" },
    {"$lookup" :
     {
            "from" : "dynasties",
            "localField" : "spouse_data.dnt",
            "foreignField" : "_id",
            "as" : "spouse_dyn"
        }
    },
    {"$unwind" : "$spouse_dyn" },
    {
        "$group":
        {
            "_id" : "$spouse_dyn.name",
            "total" : {"$sum" : 1}
        }
    },
    {"$sort" : { "total" : -1 } },
    #{"$limit" : 10}
]

In [22]:
karl = characters.aggregate(pipeline)
for char in karl:
    print(char)

{'_id': 'Karling', 'total': 49}
{'_id': 'du Perche', 'total': 26}
{'_id': 'Vuodi', 'total': 14}
{'_id': 'de Preuilly', 'total': 12}
{'_id': 'Alachisling', 'total': 11}
{'_id': 'de Trith', 'total': 9}
{'_id': 'de Flotte', 'total': 9}
{'_id': 'Ulfing', 'total': 9}
{'_id': 'Abbonid', 'total': 9}
{'_id': 'Eóganacht-Locha Léin', 'total': 9}
{'_id': 'de Lagery', 'total': 8}
{'_id': 'Nibelunging', 'total': 6}
{'_id': 'Uuidoch', 'total': 6}
{'_id': 'de Lyon', 'total': 5}
{'_id': 'de Soissons', 'total': 5}
{'_id': 'Rorgonides', 'total': 4}
{'_id': 'de Lusignan', 'total': 4}
{'_id': 'de Hainault', 'total': 4}
{'_id': 'Cerdicing', 'total': 3}
{'_id': "de l'Aigle", 'total': 3}
{'_id': 'Munichingi', 'total': 3}
{'_id': 'Erenolding', 'total': 3}
{'_id': 'de Chaumontois', 'total': 3}
{'_id': 'Capet', 'total': 3}
{'_id': 'Mel', 'total': 3}
{'_id': 'Emichid', 'total': 3}
{'_id': 'de Namur', 'total': 3}
{'_id': 'Ua Flaithbertaig', 'total': 3}
{'_id': 'Iceling', 'total': 3}
{'_id': 'Khunzakhal', 'total':